# Imports

In [ ]:
import torch
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from collections import defaultdict, Counter

In [ ]:
import matplotlib
matplotlib.rcParams['figure.facecolor'] = 'white'

In [ ]:
import pandas as pd
pd.options.display.max_columns = None

In [ ]:
%run ../mimic_cxr.py

# Load metadata

In [ ]:
FNAME_PREFIX = 'mimic-cxr-2.0.0'

In [ ]:
fpath = os.path.join(DATASET_DIR, f'{FNAME_PREFIX}-metadata.csv')
metadata = pd.read_csv(fpath)
print(len(metadata), len(metadata['dicom_id'].unique()))
metadata.head(2)

In [ ]:
fpath = os.path.join(DATASET_DIR, f'{FNAME_PREFIX}-chexpert.csv')
chexpert_df = pd.read_csv(fpath)
chexpert_df.fillna(0, inplace=True)
chexpert_df.replace(-1, 1, inplace=True)
chexpert_df.head()

In [ ]:
fpath = os.path.join(DATASET_DIR, 'reports', 'reports_with_chexpert_labels.csv')
chexpert_df_v2 = pd.read_csv(fpath)
print(len(chexpert_df_v2))
chexpert_df_v2.head(2)

In [ ]:
# To resemble the original chexpert_df dataframe:
chexpert_df_v2['subject_id'] = [
    int(row['filename'][5:13]) for _, row in chexpert_df_v2.iterrows()
]
chexpert_df_v2['study_id'] = [
    int(row['filename'][15:23]) for _, row in chexpert_df_v2.iterrows()
]
chexpert_df_v2.replace({-2: 0, -1: 1}, inplace=True)
chexpert_df_v2.head(2)

## Check repeated images

In [ ]:
keys = ['subject_id', 'study_id', 'ViewPosition', 'StudyDate', 'StudyTime']
df2 = metadata.merge(metadata, on=keys, how='left')
df2 = df2.loc[df2['dicom_id_x'] != df2['dicom_id_y']]
cols = keys + ['dicom_id_x', 'dicom_id_y', 'Rows_x', 'Columns_x', 'Rows_y', 'Columns_y']
df2 = df2[cols]
print(len(df2))
df2.head(2)

In [ ]:
unique_pairs = set()
should_keep = list()
for index, row in df2.iterrows():
    d1 = row['dicom_id_x']
    d2 = row['dicom_id_y']
    
    pair = ','.join(sorted([d1, d2]))
    if pair not in unique_pairs:
        should_keep.append(index)
        unique_pairs.add(pair)
len(should_keep)

In [ ]:
df2 = df2.loc[should_keep]
df2.head(3)

In [ ]:
len(df2), len(df2['dicom_id_x'].unique()), len(df2['subject_id'].unique())

In [ ]:
from PIL import Image

In [ ]:
def plot_repeated_sample(index):
    row = df2.iloc[index]
    
    subject_id = f'p{row["subject_id"]}'
    study_id = f's{row["study_id"]}'
    subfolder = subject_id[:3]
    
    print('Subject: ', subject_id)
    print('Position: ', row['ViewPosition'])
    
    fpath_x = f'{row["dicom_id_x"]}.jpg'
    fpath_y = f'{row["dicom_id_y"]}.jpg'

    fpath_x = os.path.join(DATASET_DIR, 'images', subfolder, subject_id, study_id, fpath_x)
    fpath_y = os.path.join(DATASET_DIR, 'images', subfolder, subject_id, study_id, fpath_y)
    
    image_x = Image.open(fpath_x)
    image_y = Image.open(fpath_y)
    print(image_x.size, image_x.mode)
    print(image_y.size, image_y.mode)
    
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.imshow(image_x, cmap='gray')

    plt.subplot(1, 2, 2)
    plt.imshow(image_y, cmap='gray')

In [ ]:
plot_repeated_sample(-21)

# Plot example

In [ ]:
from PIL import Image

In [ ]:
idx = -1001
row = metadata.iloc[idx]
row

In [ ]:
subject_id = f'p{row["subject_id"]}'
study_id = f's{row["study_id"]}'
dicom_id = str(row['dicom_id'])
image_fname = f'{dicom_id}.jpg'
subfolder = subject_id[:3]
subfolder, subject_id, study_id, image_fname

In [ ]:
name = os.path.join(subfolder, subject_id, study_id, image_fname)
# name = 'p10/p10423865/s54322560/105f82f9-a511d520-332232d1-82563ed2-42ea685b.jpg'

In [ ]:
image_fpath = os.path.join(DATASET_DIR, 'images', name)

In [ ]:
image = Image.open(image_fpath)
print(image.size, image.mode)
plt.imshow(image, cmap='gray')

In [ ]:
report_fpath = os.path.join(DATASET_DIR, 'raw-reports', subfolder, subject_id, f'{study_id}.txt')

In [ ]:
with open(report_fpath) as f:
    text = f.read()
print(text)

# Exploratory analysis

## Images by patient

In [ ]:
DATASET_DIR

In [ ]:
n_images_by_patient = Counter(metadata['subject_id'])
n_distinct_patients = len(n_images_by_patient)
n_distinct_patients, len(metadata)

### Plot distribution

In [ ]:
patients_and_amounts = sorted(n_images_by_patient.items(), key=lambda x: x[1], reverse=True)
patients_and_amounts[:10]

In [ ]:
patients_ids, amounts = zip(*patients_and_amounts)
plt.plot(range(len(amounts)), amounts)
plt.xlabel('Patient ID')
plt.ylabel('N images')
plt.title('N images by patient (MIMIC)')

In [ ]:
amounts = np.array(amounts)
cum_amounts = np.cumsum(amounts) / len(metadata)
total_patients = len(amounts)

In [ ]:
plt.plot(np.array(range(len(cum_amounts))) / total_patients, cum_amounts)
plt.title('N images vs N patients')
plt.ylabel('Fraction of images')
plt.xlabel('Fraction of patients')

In [ ]:
for perc in [0.2, 0.3, 0.4, 0.45, 0.5, 0.7, 0.8]:
    n_patients = np.argmax(cum_amounts > perc) + 1
    perc_patients = n_patients / total_patients * 100
    
    s1 = f'Top {n_patients:,} patients ({perc_patients:.1f}%)'
    s2 = f'{int(perc*100)}% of the images'
    print(f'{s1:<30} account for {s2}')
print(f'Total patients: {total_patients:,}')

### Projection by type of patient

In [ ]:
metadata.fillna('--', inplace=True)
metadata.replace('LL', 'LATERAL', inplace=True)

In [ ]:
def plot_proj_in_subset(condition, title):
    patients = [patient for patient, amount in n_images_by_patient.items() if condition(amount)]

    df = metadata
    d = df.loc[df['subject_id'].isin(set(patients))]
    images_by_proj = Counter(d['ViewPosition'])

    images_by_proj = {
        k: images_by_proj[k]
        for k in ['PA', 'AP', 'LATERAL']
    }
    
    plt.bar(images_by_proj.keys(), images_by_proj.values())
    plt.ylabel('N images')
    plt.title(title)
    
    return patients

In [ ]:
THRESH = 100

plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
pat1 = plot_proj_in_subset(lambda x: x > THRESH, f'Patients with > {THRESH}')

plt.subplot(1, 2, 2)
pat2 = plot_proj_in_subset(lambda x: x <= THRESH, f'Patients with <= {THRESH}')

# Pre-process

i.e. run only once

## Create master csv

Containing image_id, study_id, filenames and chexpert labels (by image)

### Keep only studies with a report present

In [ ]:
%run -n ../preprocess/mimic_cxr.py

In [ ]:
reports_df = load_raw_reports_df()
reports_df.head()

In [ ]:
studies_with_report = set(int(report[1:]) for report in reports_df['study'])
len(studies_with_report)

### Check metadata vs chexpert discrepancies

* Only studies with a report are kept
* There are a few studies with no report, present in metadata and chexpert_df csvs

In [ ]:
# CHOSEN_CHEXPERT = chexpert_df
CHOSEN_CHEXPERT = chexpert_df_v2

In [ ]:
len(metadata), len(CHOSEN_CHEXPERT)

In [ ]:
studies1 = set(metadata['study_id'])
studies2 = set(CHOSEN_CHEXPERT['study_id'])
assert studies2.issubset(studies1)
len(studies1), len(studies2)

In [ ]:
studies11 = studies1.intersection(studies_with_report)
studies22 = studies2.intersection(studies_with_report)
assert studies11 == studies22
len(studies11)

### Merge metadata and chexpert

In [ ]:
metadata_filtered = metadata.loc[metadata['study_id'].isin(studies_with_report)]
len(metadata_filtered), len(set(metadata_filtered['study_id']))

In [ ]:
chexpert_filtered = CHOSEN_CHEXPERT.loc[CHOSEN_CHEXPERT['study_id'].isin(studies_with_report)]
assert len(chexpert_filtered) == len(set(chexpert_filtered['study_id']))
len(chexpert_filtered)

In [ ]:
studies1 = set(metadata_filtered['study_id'])
studies2 = set(chexpert_filtered['study_id'])
assert studies1 == studies2

In [ ]:
master_df = metadata_filtered.merge(
    chexpert_filtered, on=['study_id', 'subject_id'], how='inner')
len(master_df), len(set(master_df['study_id']))

In [ ]:
master_df.head(2)

### Merge with split

In [ ]:
fpath = os.path.join(DATASET_DIR, f'{FNAME_PREFIX}-split.csv')
split_df = pd.read_csv(fpath)
split_df.head()

In [ ]:
master_df = master_df.merge(split_df, on=['dicom_id', 'study_id', 'subject_id'], how='inner')
len(master_df), len(set(master_df['study_id']))

In [ ]:
master_df.head(2)

### Add filepaths

In [ ]:
def get_filenames(subject_id, study_id, dicom_id):
    subject_id = f'p{subject_id}'
    study_id = f's{study_id}'
    image_fname = f'{dicom_id}.jpg'
    subfolder = subject_id[:3]

    image_fpath = os.path.join(subfolder, subject_id, study_id, image_fname)
    report_fpath = os.path.join(subfolder, subject_id, f'{study_id}.txt')
    
    return image_fpath, report_fpath

In [ ]:
image_fpaths, report_fpaths = zip(*[
    get_filenames(*ids)
    for ids in zip(master_df['subject_id'], master_df['study_id'], master_df['dicom_id'])
])
len(image_fpaths), len(report_fpaths)

In [ ]:
master_df['image_fpath'] = image_fpaths
master_df['report_fpath'] = report_fpaths

In [ ]:
master_df.head(2)

### Add report_length (chexpert_v2)

In [ ]:
if 'Reports' in master_df:
    master_df['report_length'] = = [
        len(row['Reports'].split())
        for _, row in master_df.iterrows()
    ]
master_df.head(3)

### Check lateral or frontal

#### Check weird positions 

In [ ]:
positions = list(Counter(master_df['ViewPosition']).keys())
positions

In [ ]:
weird_pos = positions[5:]
weird_pos

In [ ]:
cols = ['image_fpath',
        'ViewPosition', 'PerformedProcedureStepDescription', 'ViewCodeSequence_CodeMeaning',
        # 'dicom_id',
       ]
df = master_df[cols]
weird_images = df.loc[df['ViewPosition'].isin(weird_pos)]
print(len(weird_images))

In [ ]:
weird_images.sort_values('ViewPosition')

In [ ]:
' '.join(list(weird_images['image_fpath']))

In [ ]:
FRONTAL_POSITIONS = [
    'PA', 'AP',
    ### These were manually reviewed and are frontal positions:
    'AP AXIAL', 'LAO', 'LPO', 'RAO',
]

#### Check nan positions

FIXME: For now, samples with ViewPosition == nan, maybe frontal or lateral

In [ ]:
Counter(master_df['ViewPosition'])

In [ ]:
cols = ['image_fpath',
        'ViewPosition', 'PerformedProcedureStepDescription', 'ViewCodeSequence_CodeMeaning',
        # 'dicom_id',
       ]
df = master_df[cols]
nan_positions = df.loc[df['ViewPosition'].isnull()]
len(nan_positions)

In [ ]:
nan_positions['ViewCodeSequence_CodeMeaning'].isnull().sum()

In [ ]:
' '.join(list(nan_positions['image_fpath'])[:10])

### Save master csv

In [ ]:
%run ../common/__init__.py

In [ ]:
cols = ['dicom_id', 'subject_id', 'study_id',
        'image_fpath', 'report_fpath',
        'ViewPosition', 'split',
        ]
if 'report_length' in master_df:
    cols.append('report_length')
cols.extend(CHEXPERT_DISEASES)

In [ ]:
out_df = master_df[cols]
out_df.replace('validate', 'val', inplace=True)
out_df.head()

In [ ]:
len(out_df), len(set(out_df['subject_id'])), len(set(out_df['study_id']))

In [ ]:
Counter(out_df['split'])

In [ ]:
raise Exception('Check where you are saving first!!')
# fpath = os.path.join(DATASET_DIR, 'master_metadata.v4-2-fixed.csv')
# out_df.to_csv(fpath, index=False)

## Clean and tokenize reports

### Review reports and tokens manually

In [ ]:
import re

In [ ]:
%run ../../utils/nlp.py
%run ../mimic_cxr.py

In [ ]:
reports_fname = os.path.join(DATASET_DIR, 'reports', 'reports.clean.v1.json')
with open(reports_fname, 'r') as f:
    reports = json.load(f)

In [ ]:
token_appearances = Counter()
for r in reports.values():
    for token in r['clean_text'].split():
        token_appearances[token] += 1
len(token_appearances)

In [ ]:
more_than_k_appearances = lambda x: [(k, v) for k, v in token_appearances.items() if v > x]

In [ ]:
len(more_than_k_appearances(10))

#### Vocab

In [ ]:
[(k, v) for k, v in token_appearances.items() if re.search(r'twe', k)]

In [ ]:
sorted(more_than_k_appearances(10), key=lambda x: x[1], reverse=False)

#### Reports

In [ ]:
def find_reports_with_tokens(tokens, search_in='clean_text', absent_in=None):
    if isinstance(tokens, str):
        tokens = [tokens]
    found = []
    for report in reports.values():
        text = report[search_in]
        
        for token in tokens:
            if re.search(token, text):
                if absent_in is not None:
                    if not re.search(token, report[absent_in]):
                        continue
                found.append(report)
                break
                
    return found

In [ ]:
found = find_reports_with_tokens(r'\b[kjhv]\b', absent_in='text')
len(found)

In [ ]:
found

#### Sentences

In [ ]:
sentences_appears = get_sentences_appearances(r['clean_text'] for r in reports.values())
sentences = list(sentences_appears)
len(sentences)

In [ ]:
sentences[100:200]

In [ ]:
found = [s for s in sentences if re.search(r'\A\W', s)]
len(found)

In [ ]:
found

### Actually debug tokenizer

In [ ]:
%run ../preprocess/tokenize.py

In [ ]:
ts = text_to_tokens('M.D.')
ts

### Run preprocess

In [ ]:
%run -n ../preprocess/mimic_cxr.py

In [ ]:
# Takes around 1min
reports, token_appearances, errors = preprocess_mimic_cxr(
    'v4-2', override=True, target_keys=['findings', 'text'])
len(reports), len(token_appearances)

In [ ]:
# TODO: pass studies=mini_studies to save a version with only reports from mini-mimic

## took ~6seconds
df_organs, errors = create_sentences_with_organs('v4', show=True)
df_organs.head()

### Check errors

In [ ]:
REPORTS_DF = load_raw_reports_df()
REPORTS_DF.head()

In [ ]:
fpath = os.path.join(DATASET_DIR, 'master_metadata.csv')
master_df = pd.read_csv(fpath)
master_df.head(2)

In [ ]:
def load_raw_report(study_id):
    if isinstance(study_id, str):
        study_id = int(study_id.strip('s'))

    d = master_df.loc[master_df['study_id'] == study_id]
    
    report_fpaths = list(d['report_fpath'].unique())
    assert len(report_fpaths) == 1, f'Not 1 subject: {report_fpaths}'
    report_fpath = report_fpaths[0]
    print(report_fpath)
    report_fpath = os.path.join(DATASET_DIR, 'raw-reports', report_fpath)
    
    with open(report_fpath) as f:
        text = f.read()
    
    return text

In [ ]:
studies = list(f's{s}' for s in errors['tokens-empty'])
len(studies)

In [ ]:
for study_id in studies:
    print('=' * 60)
    print('Study ID: ', study_id)

    report = load_raw_report(study_id)
    print(report)

    print('-' * 30)

    d = REPORTS_DF.loc[REPORTS_DF['study'] == study_id]
    ids = list(d.index)
    assert len(ids) == 1, f'Not 1 study: {d}'
    d = d.loc[ids[0]]
    for k in ['text', 'comparison', 'findings', 'impression', 'last_paragraph']:
        print(f'{k}: {d[k]}')

    print('=' * 60)

## Calculate mean and std

In [ ]:
%run ../mimic_cxr.py
%run ../../utils/images.py

In [ ]:
fpath = os.path.join(DATASET_DIR, 'master_metadata.csv')
d = pd.read_csv(fpath)
d = d.loc[d['split'] == 'train']
train_images = list(d['image_fpath'].unique())
len(train_images)

In [ ]:
image_folder = os.path.join(DATASET_DIR, 'images')

In [ ]:
%%capture output
%%time

mean, std = compute_mean_std(ImageFolderIterator(image_folder, train_images), show=True)
mean, std

In [ ]:
mean, std

### Check empty ones

In [ ]:
error_appearances = {
    sentence: sentences_appears[sentence]
    for sentence in errors['all-empty']
}
n_errors = sum(error_appearances.values())
perc = n_errors / total_appearances * 100
f'{n_errors:,}', f'{total_appearances:,}', f'{perc:.0f}%'

In [ ]:
l = sorted(error_appearances.items(), key=lambda x: x[1], reverse=True)
l[:10]

### Save sentence2organ in file

In [ ]:
%run ../../utils/__init__.py
%run ../common/constants.py

In [ ]:
from medai.datasets.iu_xray import DATASET_DIR as IU_DATASET_DIR

In [ ]:
fpath = os.path.join(IU_DATASET_DIR, 'reports', 'sentences_with_organs_OLD.csv')
iu_df = pd.read_csv(fpath)
iu_df.head()

In [ ]:
final_df = iu_df.append(mimic_df, ignore_index=True)
len(final_df), len(iu_df) + len(mimic_df)

In [ ]:
final_df = final_df.groupby('sentences').last()
final_df.reset_index(drop=False, inplace=True)
print(len(final_df))
final_df.head()

In [ ]:
fpath = os.path.join(WORKSPACE_DIR, 'sentences_with_organs.csv')
final_df.to_csv(fpath, index=False)

## Apply chexpert labeler

### Option 1: use available labels

MIMIC reports already come with Chexpert labels, so all there is to do is adapt the CSV file

#### Load original file

In [ ]:
fname = os.path.join(DATASET_DIR, 'mimic-cxr-2.0.0-chexpert.csv')

original_df = pd.read_csv(fname)
original_df.head()

#### Add full filepaths

In [ ]:
def get_report_fpaths(subject_id, study_id):
    """Given subject and study ids, returns the report filepath."""
    subject_id = f'p{subject_id}'
    study_id = f's{study_id}'
    subfolder = subject_id[:3]

    report_fpath = os.path.join(subfolder, subject_id, f'{study_id}.txt')
    
    return report_fpath

In [ ]:
filenames = [
    get_report_fpaths(subject_id, study_id)
    for subject_id, study_id in zip(original_df['subject_id'], original_df['study_id'])
]
len(filenames), len(original_df)

In [ ]:
original_df['filename'] = filenames
original_df.head()

#### Add cleaned reports

In [ ]:
fname = os.path.join(DATASET_DIR, 'reports', 'reports.clean.v4-2.json')
with open(fname, 'r') as f:
    clean_reports = json.load(f)
len(clean_reports)

In [ ]:
original_df = original_df.loc[original_df['study_id'].isin(set(clean_reports))]
len(original_df)

In [ ]:
reports = [
    re.sub(r'\s+', ' ', clean_reports[str(study_id)]['clean_text'])
    for study_id in original_df['study_id']
]
len(reports), len(original_df)

In [ ]:
original_df['Reports'] = reports
original_df.head()

#### Save to file

In [ ]:
cols = ['filename', 'Reports'] + CHEXPERT_DISEASES
adapted_df = original_df[cols]
adapted_df.fillna(-2, inplace=True)
adapted_df = adapted_df.astype({c:np.int8 for c in CHEXPERT_DISEASES})
adapted_df.head()

In [ ]:
fpath = os.path.join(DATASET_DIR, 'reports', 'reports_with_chexpert_labels.csv')
if False and os.path.isfile(fpath):
    raise Exception('Overriding file!')
adapted_df.to_csv(fpath, index=False)

### Option 2: calculate again

* Some findings/impression sections are poorly extracted, but the chexpert-labels were calculated for the full report --> mismatch!!
* E.g.: extracted findings section is: "is a .", but the chexpert-labels are positive for some abnormalities.
* Solution: apply the labeler again! Takes a lot of time, but run only once

In [ ]:
# %run ../preprocess/mimic_cxr_chexpert.py

In [ ]:
### Read chexpert labels to check them out!
ACTUAL_DISEASES = CHEXPERT_DISEASES[1:]

In [ ]:
df = pd.read_csv(os.path.join(DATASET_DIR, 'reports', 'reports_with_chexpert_labels.csv'))
df = df.replace({ -2: 0 })
df.head(2)

In [ ]:
d = df
d = d.loc[(d[ACTUAL_DISEASES] == 0).all(axis=1)]
d.head(2)

In [ ]:
list(d['Reports'])

In [ ]:
d.loc[d['Reports'].str.startswith('no acute')]

# Create mini-mimic

Mini-mimic stats:

* Total images: 43,590, train: 42,435, val: 430, test: 725
* Frontal images: 29,189, train: 28,380, val: 289, test: 520

In [ ]:
FPATH = os.path.join(DATASET_DIR, 'master_metadata.csv')
df = pd.read_csv(FPATH)
print(len(df))
df.head()

## Create random split

In [ ]:
is_mini = list()
percentages = {
    'train': 0.12,
    'val': 0.15,
    'test': 0.15,
}

for split in ('train', 'val', 'test'):
    sub_df = df.loc[df['split'] == split]
    
    # Do not use images without frontal-lateral definition
    sub_df = sub_df.dropna(axis=0, subset=['ViewPosition'])
    
    images = list(sub_df['dicom_id'])
    k = int(percentages[split] * len(images))
    
    print(f'Choosing {k:,} from {split}')
    is_mini.extend(random.sample(images, k))
len(is_mini)

## Add column to master_df

In [ ]:
is_mini = set(is_mini)

is_mini_column = [
    int(dicom_id in is_mini)
    for dicom_id in df.dicom_id
]
len(is_mini), len(is_mini_column)

In [ ]:
df['mini'] = is_mini_column

In [ ]:
df.to_csv(FPATH, index=False)

# Move mini-mimic to SSD

In [ ]:
import os
from tqdm.auto import tqdm

In [ ]:
FPATH = os.path.join(DATASET_DIR, 'master_metadata.csv')
# df = pd.read_csv(FPATH)
print(len(df))
df.head()

In [ ]:
df = df[df.mini == 1]
len(df)

## Sum space needed

In [ ]:
total_size = 0

images_dir = os.path.join(DATASET_DIR, 'images')
for image_path in tqdm(df.image_fpath):
    fpath = os.path.join(images_dir, image_path)
    r = os.stat(fpath)
    total_size += r.st_size / 1024 # kbytes
    
total_size /= 1024 # mbytes
total_size

In [ ]:
total_size / 1024

## Copy images

In [ ]:
from shutil import copyfile

In [ ]:
src_dir = os.path.join(DATASET_DIR, 'images')
target_dir = os.path.join(DATASET_DIR_FAST, 'images')
for image_path in tqdm(df.image_fpath):
    src_fpath = os.path.join(src_dir, image_path)
    target_fpath = os.path.join(target_dir, image_path)
    os.makedirs(os.path.dirname(target_fpath), exist_ok=True)
    copyfile(src_fpath, target_fpath)

# Create images-small

In [ ]:
from PIL import Image
from tqdm.auto import tqdm

In [ ]:
TARGET_SIZE = (256, 256)

In [ ]:
%%time

pbar = tqdm(total=377110)

for root, dirs, filenames in os.walk(images_src):
    for filename in filenames:
        # Original image
        src_path = os.path.join(root, filename)
        
        # Small image
        dest_folder = root.replace('/images/', '/images-small/')
        os.makedirs(dest_folder, exist_ok=True)
        dest_path = os.path.join(dest_folder, filename)

        if os.path.isfile(dest_path):
            pbar.update(1)
            continue

        src_image = Image.open(src_path)
        dest_image = src_image.resize(TARGET_SIZE)
        
        dest_image.save(dest_path)
        src_image.close()
        
        pbar.update(1)

In [ ]:
print(output)

# Debug Dataset class

In [ ]:
%run ../mimic_cxr.py
%run ../../utils/common.py

In [ ]:
dataset = MIMICCXRDataset('val', sort_samples=True, image_size=(256, 256))
len(dataset), len(dataset.reports)

In [ ]:
%%time

dataset = MIMICCXRDataset('train', sort_samples=True)
len(dataset), len(dataset.reports)

In [ ]:
item = dataset[4]
item.image.size(), item.report

In [ ]:
item.image.min(), item.image.max()

In [ ]:
len(dataset.reports)

In [ ]:
plt.imshow(tensor_to_range01(item.image).permute(1, 2, 0))

# Plot distributions

In [ ]:
%run ../mimic_cxr.py
%run ../../utils/common.py

In [ ]:
train_dataset = MIMICCXRDataset('train')
train_dataset_mini = MIMICCXRDataset('train', mini=1)
val_dataset = MIMICCXRDataset('val')
val_dataset_mini = MIMICCXRDataset('val', mini=1)
len(train_dataset), len(train_dataset_mini), len(val_dataset), len(val_dataset_mini)

In [ ]:
def get_dataset_name(dataset):
    if dataset._mini is None:
        return dataset.dataset_type
    return f'mini{dataset._mini}-{dataset.dataset_type}'

## Labels distributions

In [ ]:
def plot_labels_distribution(dataset):
    amounts_by_disease = dataset.master_df[CHEXPERT_DISEASES].sum(
        axis=0).sort_values(ascending=False)

    plt.title(get_dataset_name(dataset), fontsize=20)
    plt.bar(amounts_by_disease.index, amounts_by_disease.values)
    plt.xticks(rotation=60, fontsize=15, ha='right')
    plt.ylabel('Amount of samples', fontsize=15)
    plt.ylim(0, max(amounts_by_disease) * 1.15)
    y_padding = int(max(amounts_by_disease) * 0.03)
    
    n_samples = len(dataset.master_df)
    for index, (disease, amount) in enumerate(amounts_by_disease.iteritems()):
        amount = int(amount)
        perc = amount / n_samples * 100
        plt.text(index, amount + y_padding, f'{amount:,}\n{perc:.0f}%', ha='center')

In [ ]:
n_rows = 2
n_cols = 2
plt.figure(figsize=(15, n_rows * 8))

plt.subplot(n_rows, n_cols, 1)
plot_labels_distribution(train_dataset)

plt.subplot(n_rows, n_cols, 2)
plot_labels_distribution(val_dataset)

plt.subplot(n_rows, n_cols, 3)
plot_labels_distribution(train_dataset_mini)

plt.subplot(n_rows, n_cols, 4)
plot_labels_distribution(val_dataset_mini)

plt.tight_layout()

## Length distribution

In [ ]:
def plot_n_words_distribution(dataset):
    df = dataset.master_df.groupby('report_fpath').first()
    lengths = df['report_length']
    plt.title(f'Distribution of report-length ({get_dataset_name(dataset)})')
    plt.ylabel('Amount of images')
    plt.xlabel('Number of words')
    _ = plt.hist(lengths.values, bins=25, range=(0,200))

In [ ]:
n_rows = 2
n_cols = 2

plt.figure(figsize=(7 * n_cols, 5*n_rows))
plt.subplot(n_rows, n_cols, 1)
plot_n_words_distribution(train_dataset)

plt.subplot(n_rows, n_cols, 2)
plot_n_words_distribution(train_dataset_mini)

plt.subplot(n_rows, n_cols, 3)
plot_n_words_distribution(val_dataset)

plt.subplot(n_rows, n_cols, 4)
plot_n_words_distribution(val_dataset_mini)

## Frontal vs lateral distribution

In [ ]:
def plot_view_position_distribution(dataset):
    amounts = Counter(dataset.master_df['ViewPosition'])
    
    reduced_amounts = Counter()
    for key, value in amounts.items():
        key = str(key)
        if key == 'nan':
            reduced_key = 'nan'
        elif key in _FRONTAL_POSITIONS:
            reduced_key = 'frontal'
        else:
            reduced_key = 'lateral'
            
        reduced_amounts[reduced_key] += value
    
    plt.title(f'Frontal vs lateral ({get_dataset_name(dataset)})')
    plt.ylabel('Amount of images')
    # plt.xticks(rotation=90)
    
    reduced_amounts = sorted(reduced_amounts.items(), key=lambda x: x[1], reverse=True)
    keys, values = zip(*reduced_amounts)
    plt.bar(keys, values)
    
    plt.ylim(0, max(values) * 1.2)
    y_padding = max(values) * 0.03
    n_samples = len(dataset)
    for index, value in enumerate(values):
        perc = value / n_samples * 100
        text = f'{value:,}\n{perc:.2f}%'
        plt.text(index, value + y_padding, text, ha='center')

In [ ]:
n_rows = 2
n_cols = 2

plt.figure(figsize=(n_cols * 7, n_rows * 5))

plt.subplot(n_rows, n_cols, 1)
plot_view_position_distribution(train_dataset)
plt.subplot(n_rows, n_cols, 2)
plot_view_position_distribution(train_dataset_mini)

plt.subplot(n_rows, n_cols, 3)
plot_view_position_distribution(val_dataset)
plt.subplot(n_rows, n_cols, 4)
plot_view_position_distribution(val_dataset_mini)